# Ensemble v2 - Only Valid Submissions

Scan only submission.csv files in submission folders (which should be valid) and compare with baseline.

In [ ]:
import numpy as np
import pandas as pd
import math
from numba import njit
import glob
import os
from tqdm import tqdm
import json
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125], dtype=np.float64)
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5], dtype=np.float64)

In [ ]:
@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

# Warm up numba
_ = score_group(np.array([0.0]), np.array([0.0]), np.array([45.0]), TX, TY)
print("Numba warmed up")

In [ ]:
# Find only submission.csv files in submission folders (these should be valid)
submission_files = glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/submission.csv')
print(f"Found {len(submission_files)} submission files")

# Also check the baseline
baseline_path = '/home/nonroot/snapshots/santa-2025/21322576451/submission/submission.csv'
print(f"Baseline: {baseline_path}")

In [ ]:
# Load baseline and calculate scores for each N
baseline_df = pd.read_csv(baseline_path)
baseline_df['N'] = baseline_df['id'].astype(str).str.split('_').str[0].astype(int)

best = {}
for n, g in baseline_df.groupby('N'):
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    sc = score_group(xs, ys, ds, TX, TY)
    best[n] = {'score': float(sc), 'data': g.drop(columns=['N']).copy(), 'src': 'baseline'}

baseline_total = sum(best[n]['score'] for n in range(1, 201))
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Scan all submission files and look for improvements
improved_count = 0
for fp in tqdm(submission_files, desc="Scanning submissions"):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    df = df.copy()
    try:
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    except:
        continue
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        if len(g) != n:
            continue
        
        try:
            xs = strip(g['x'].to_numpy())
            ys = strip(g['y'].to_numpy())
            ds = strip(g['deg'].to_numpy())
            sc = score_group(xs, ys, ds, TX, TY)
            
            if sc < best[n]['score'] - 1e-9:
                best[n]['score'] = float(sc)
                best[n]['data'] = g.drop(columns=['N']).copy()
                best[n]['src'] = fp.split('/')[-3]  # snapshot ID
                improved_count += 1
        except:
            continue

print(f"\nFound {improved_count} improvements over baseline")

In [ ]:
# Override N=1 with optimal 45 degree rotation
manual_data = pd.DataFrame({
    'id': ['001_0'],
    'x': ['s0.0'],
    'y': ['s0.0'],
    'deg': ['s45.0']
})
xs = strip(manual_data['x'].to_numpy())
ys = strip(manual_data['y'].to_numpy())
ds = strip(manual_data['deg'].to_numpy())
sc = score_group(xs, ys, ds, TX, TY)
if sc < best[1]['score']:
    best[1]['score'] = float(sc)
    best[1]['data'] = manual_data.copy()
    best[1]['src'] = 'optimal_45deg'
    print(f"N=1 improved to optimal 45° rotation, score: {sc:.6f}")
else:
    print(f"N=1 already optimal, score: {best[1]['score']:.6f}")

In [ ]:
# Build ensemble
rows = []
total_score = 0.0
source_counts = {}

for n in range(1, 201):
    entry = best[n]
    rows.append(entry['data'])
    total_score += entry['score']
    src = entry['src']
    source_counts[src] = source_counts.get(src, 0) + 1

print(f"Total ensemble score: {total_score:.6f}")
print(f"Improvement over baseline: {baseline_total - total_score:.6f}")
print(f"\nSources:")
for src, count in sorted(source_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {src}: {count} N values")

In [ ]:
# Concatenate and sort
ensemble_df = pd.concat(rows, ignore_index=True)
ensemble_df['sn'] = ensemble_df['id'].str.split('_').str[0].astype(int)
ensemble_df['si'] = ensemble_df['id'].str.split('_').str[1].astype(int)
ensemble_df = ensemble_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
ensemble_df = ensemble_df[['id', 'x', 'y', 'deg']]

print(f"Ensemble has {len(ensemble_df)} rows")
print(f"Expected: {sum(range(1, 201))} rows")

In [ ]:
# Save ensemble
ensemble_df.to_csv('/home/code/experiments/002_ensemble/ensemble_valid.csv', index=False)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print("Saved ensemble")

# Save metrics
metrics = {'cv_score': total_score}
with open('/home/code/experiments/002_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")

In [ ]:
# Score breakdown
print("\nScore breakdown by N range:")
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(best[n]['score'] for n in range(start, end+1))
    pct = range_score / total_score * 100
    print(f"  N={start}-{end}: {range_score:.4f} ({pct:.1f}%)")